In [1]:
from models.waffleiron.segmenter import Segmenter
import torch
from datasets import LIST_DATASETS, Collate
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import argparse
import wandb
from torchmetrics.classification import MulticlassJaccardIndex

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

import torchhd
from torchhd.models import Centroid
from torchhd import embeddings

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))
device_string = "cuda:0" if torch.cuda.is_available() else "cpu"

model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=16, # class for prediction
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

# Load pretrained model
ckpt = torch.load('/root/main/ScaLR/saved_models/ckpt_last_scalr.pth', map_location=device_string) # cuda:0
ckpt = ckpt["net"]

new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        if k.startswith("module.classif.0"):
            continue
        elif k.startswith("module.classif.1"):
            new_ckpt["classif" + k[len("module.classif.1") :]] = ckpt[k]
        else:
            new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]

model.load_state_dict(new_ckpt)

if device_string != 'cpu':
    torch.cuda.set_device(device_string) # cuda:0
    model = model.cuda(device_string) # cuda:0

model.eval()

kwargs = {
        "rootdir": '/root/main/dataset/nuscenes',
        "input_feat": ["xyz", "intensity", "radius"],
        "voxel_size": 0.1,
        "num_neighbors": 16,
        "dim_proj": [2, 1, 0],
        "grids_shape": [[256, 256], [256, 32], [256, 32]],
        "fov_xyz": [[-64, -64, -8], [64, 64, 8]], # Check here
    }

# Get datatset
DATASET = LIST_DATASETS.get("nuscenes")

# Train dataset
train_dataset = DATASET(
    phase="train",
    **kwargs,
)

test_dataset = DATASET(
    phase="val",
    **kwargs,
)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(),
    )

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(),
    )

DIMENSIONS = 10000
FEAT_SIZE = 768
NUM_LEVELS = 1000
BATCH_SIZE = 1  # for GPUs with enough memory we can process multiple images at ones


class Encoder(nn.Module):
    def __init__(self, out_features, size, levels):
        super(Encoder, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.position = embeddings.Random(size, out_features)
        self.value = embeddings.Level(levels, out_features)

    def forward(self, x):
        sample_hv = torchhd.bind(self.position.weight, self.value(x))
        sample_hv = torchhd.multiset(sample_hv)
        return torchhd.hard_quantize(sample_hv)


encode = Encoder(DIMENSIONS, FEAT_SIZE, NUM_LEVELS)
encode = encode.to(device)

num_classes = 16
model_hd = Centroid(DIMENSIONS, num_classes)
model_hd = model_hd.to(device)

def forward_model(it, batch, stop):
    feat = batch["feat"]
    labels = batch["labels_orig"]
    cell_ind = batch["cell_ind"]
    occupied_cell = batch["occupied_cells"]
    neighbors_emb = batch["neighbors_emb"]
    if device_string != 'cpu':
        feat = feat.cuda(0, non_blocking=True)
        labels = labels.cuda(0, non_blocking=True)
        batch["upsample"] = [
            up.cuda(0, non_blocking=True) for up in batch["upsample"]
        ]
        cell_ind = cell_ind.cuda(0, non_blocking=True)
        occupied_cell = occupied_cell.cuda(0, non_blocking=True)
        neighbors_emb = neighbors_emb.cuda(0, non_blocking=True)

    with torch.no_grad():
        out = model(feat, cell_ind, occupied_cell, neighbors_emb, stop)
        embed, tokens = out[0][0], out[1][0]
        embed = embed.transpose(0, 1)
        tokens = tokens.transpose(0, 1)

        labels_v = [[] for i in range(embed.shape[0])]
        for i, vox in enumerate(batch["upsample"][0]):
            labels_v[vox].append(labels[i])
        labels_v_single = []
        for labels_ in labels_v:
            lab_tens = torch.tensor(labels_)
            most_common_value = torch.bincount(lab_tens).argmax()
            labels_v_single.append(most_common_value)
    
    return tokens, labels_v_single   

Using torch.scatter_reduce for 3D to 2D projection.
Using torch.scatter_reduce for 3D to 2D projection.
Using cpu device


/tmp/ipykernel_19720/1188032203.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('/root/main/ScaLR/saved_models/ckpt_last_scalr.pth', map_location=devi

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.514 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
323
Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.749 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
stop = 0
class_count = {}
for c in range(16):
    class_count[c] = 0
for it, batch in enumerate(train_loader):
    
    # Network inputs
    
    tokens, labels_v_single = forward_model(it, batch, stop)
    labels_v_single = torch.tensor(labels_v_single)
    for c in range(16):
        class_count[c] += int(torch.bincount(labels_v_single)[c])

    print(class_count)

{0: tensor(0), 1: tensor(2), 2: tensor(424), 3: tensor(260), 4: tensor(0), 5: tensor(15), 6: tensor(174), 7: tensor(0), 8: tensor(0), 9: tensor(569), 10: tensor(4789), 11: tensor(0), 12: tensor(2887), 13: tensor(881), 14: tensor(5421), 15: tensor(3312)}
{0: tensor(0), 1: tensor(2), 2: tensor(424), 3: tensor(764), 4: tensor(0), 5: tensor(64), 6: tensor(187), 7: tensor(0), 8: tensor(0), 9: tensor(569), 10: tensor(11131), 11: tensor(14), 12: tensor(3636), 13: tensor(3207), 14: tensor(7542), 15: tensor(9513)}
{0: tensor(0), 1: tensor(2), 2: tensor(424), 3: tensor(1438), 4: tensor(0), 5: tensor(64), 6: tensor(658), 7: tensor(1), 8: tensor(0), 9: tensor(569), 10: tensor(16782), 11: tensor(14), 12: tensor(5535), 13: tensor(3207), 14: tensor(14776), 15: tensor(10025)}
{0: tensor(0), 1: tensor(2), 2: tensor(504), 3: tensor(4264), 4: tensor(0), 5: tensor(80), 6: tensor(658), 7: tensor(1), 8: tensor(0), 9: tensor(940), 10: tensor(21451), 11: tensor(14), 12: tensor(5543), 13: tensor(4417), 14: ten

KeyboardInterrupt: 

In [5]:
import torch
import torch.nn.functional as F

def gumbel_softmax_sampling(points, scores, num_samples, temperature=1.0):
    """
    Perform Gumbel-Softmax sampling to select a subset of points.
    
    Args:
        points (torch.Tensor): Tensor of shape (N, D) where N is the number of points 
                               and D is the dimensionality of each point (e.g., 3 for x, y, z).
        scores (torch.Tensor): Tensor of shape (N,) representing the importance score for each point.
        num_samples (int): Number of points to sample.
        temperature (float): Gumbel-Softmax temperature parameter.
        
    Returns:
        torch.Tensor: Sampled subset of points of shape (num_samples, D).
    """
    # Add Gumbel noise to scores
    gumbel_noise = -torch.log(-torch.log(torch.rand_like(scores)))
    noisy_scores = scores + gumbel_noise

    # Apply softmax with temperature
    probabilities = F.softmax(noisy_scores / temperature, dim=0)

    # Sample indices based on the probabilities
    sampled_indices = torch.multinomial(probabilities, num_samples, replacement=False)

    # Select and return the sampled points
    sampled_points = points[sampled_indices]
    return sampled_points

# Example usage
N = 1000  # Total number of points
D = 3     # Dimensionality of each point (x, y, z)

# Generate random points and importance scores
points = torch.randn(N, D)                # Random point cloud with N points in 3D space
scores = torch.randn(N)                   # Random scores for each point (learned in practice)

# Set parameters for Gumbel-Softmax sampling
num_samples = 100                         # Number of points to sample
temperature = 0.5                         # Temperature for Gumbel-Softmax

# Perform sampling
sampled_points = gumbel_softmax_sampling(points, scores, num_samples, temperature)

print("Sampled Points:", sampled_points)
print("Sampled Points Shape:", sampled_points.shape)

Sampled Points: tensor([[-2.1071e+00,  8.0456e-01,  5.3235e-01],
        [ 9.6123e-01,  9.5048e-01,  1.2554e+00],
        [-7.7664e-01, -1.0483e+00,  7.2708e-01],
        [-1.5455e+00, -6.3798e-02,  2.1149e-01],
        [ 9.5389e-02,  1.6103e+00,  4.3574e-01],
        [-1.5053e+00, -4.1079e-01, -4.6682e-01],
        [-6.0286e-01, -1.6169e+00, -4.6093e-01],
        [ 1.0509e+00, -2.1503e-01, -1.4380e+00],
        [ 1.3489e+00,  3.2722e-01, -4.8417e-01],
        [ 8.6167e-01, -3.4945e-02, -6.6743e-01],
        [-5.3652e-01,  1.1282e+00, -1.3726e+00],
        [ 6.5648e-02,  1.1736e+00, -1.3856e-02],
        [ 1.6718e+00, -1.5665e+00,  7.5866e-02],
        [-1.8662e+00,  4.2625e-01, -4.8351e-01],
        [ 7.2641e-01, -9.4780e-01,  8.9519e-02],
        [-1.5998e+00, -1.6832e+00,  1.3395e+00],
        [ 6.8542e-01,  5.7686e-01,  7.9222e-02],
        [ 4.8838e-01, -9.4376e-01,  1.3059e+00],
        [-8.1500e-01,  2.1610e-01, -4.9315e-01],
        [-1.5528e+00, -1.4195e+00,  3.0055e-01],
    